#1 Introducción

En el siguiente cuaderno se realiza el algoritmo que obtiene la matriz transpuesta de una matriz (su orden se ingresa por parametro) de números flotantes aplicando paralelismo y utilizando el GPGPU.

El algoritmo de este cuadernos se basa en, valga la redundancia, del algortimo para transponer una matriz proveniente del algebra lineal[3]

El objetivo es comparar los tiempos de ejecución de ambas versiones del algoritmo que obtiene la matriz transpuesta, utilizando el lenguaje python [2],la plataforma colab [1] y CUDA [4,5].

#2 Preparado del ambiente

Instala en el cuaderno el módulo CUDA de Python.

In [ ]:
!pip install pycuda

     |████████████████████████████████| 1.6MB 8.5MB/s 
     |████████████████████████████████| 71kB 11.3MB/s 
     |████████████████████████████████| 81kB 11.5MB/s 
  Created wheel for pycuda: filename=pycuda-2020.1-cp36-cp36m-linux_x86_64.whl size=621228 sha256=a4fd66bbd14df96e984ca5d363f5232c3dd1ec0807c6868240d50dd179999deb
  Stored in directory: /root/.cache/pip/wheels/8f/78/d1/5bb826f81d9d490297a348d818ff3ee6dd6f2075b06dde6ea0
  Created wheel for pytools: filename=pytools-2020.4.3-py2.py3-none-any.whl size=61374 sha256=b229c6d1185706ec703069f4c906dbfbeeb28e1f076aedb6585e38868e7432d1
  Stored in directory: /root/.cache/pip/wheels/af/c7/81/a22edb90b0b09a880468b2253bb1df8e9f503337ee15432c64
Successfully built pycuda pytools


#3 Desarrollo

In [ ]:
#@title Parametros { vertical-output: true, display-mode: "both" }
ordenMatriz =  5#@param {type:"number"}
try:
  from datetime import datetime
  tiempo_total = datetime.now()

  import pycuda.driver as cuda
  import pycuda.autoinit
  from   pycuda.compiler import SourceModule
  import numpy

  # Definición de función que transforma el tiempo en  milisegundos
  tiempo_en_ms = lambda dt:(dt.days * 24 * 60 * 60 + dt.seconds) * 1000 + dt.microseconds / 1000.0
  
  # CPU - Defino la memoria de las matrices en cpu.
  matriz = numpy.random.random((ordenMatriz,ordenMatriz))
  matriz = matriz.astype(numpy.float32())

  matrizT = numpy.empty_like(matriz)

  # CPU - reservo la memoria GPU.
  matrizGpu = cuda.mem_alloc(matriz.nbytes)
  matriztGpu = cuda.mem_alloc(matrizT.nbytes)

  # GPU - Copio la memoria al GPU.
  cuda.memcpy_htod(matrizGpu,matriz)
  cuda.memcpy_htod(matriztGpu,matrizT)

  #CPU - Defino la funcion kernel que ejecutará en GPU
  module = SourceModule("""
  __global__ void transponer(int n, float *matriz ,float *matrizT ,float *pvector1)
  {
      int idx = threadIdx.x + blockIdx.x*blockDim.x;
      int idy = threadIdx.y + blockIdx.y*blockDim.y;
      
      

      if(idx<n && idy<n)
      {
        matrizT[idy*n+idx] = matriz[idx*n+idy];
      }
  }

  """)

  kernel = module.get_function("transponer")

  tiempo_gpu = datetime.now()

  dim_hilo_x = 16
  dim_bloque_x = numpy.int( (ordenMatriz+dim_hilo_x-1) / dim_hilo_x )

  dim_hilo_y = 16
  dim_bloque_y = numpy.int( (ordenMatriz+dim_hilo_y-1) / dim_hilo_y )

  print( "Thread x: ", dim_hilo_x, ", Bloque x:", dim_bloque_x )
  print( "Thread y: ", dim_hilo_y, ", Bloque y:", dim_bloque_y )

  kernel( numpy.int32(ordenMatriz), matrizGpu ,matriztGpu, block=( dim_hilo_x, dim_hilo_y, 1 ),grid=(dim_bloque_x, dim_bloque_y,1) )

  tiempo_gpu = datetime.now() - tiempo_gpu

  cuda.memcpy_dtoh(matrizT,matriztGpu)

  tiempo_total = datetime.now() - tiempo_total

  # CPU - Informo el resultado.
  print("matriz original: ")
  print(matriz)
  print("")
  print("matriz transpuesta: ")
  print(matrizT)
  print("")
  print("Tiempo CPU: ", tiempo_en_ms( tiempo_total ), "[ms]" )
  print("Tiempo GPU: ", tiempo_en_ms( tiempo_gpu ), "[ms]" )
except ModuleNotFoundError :
      print("No se compilo el código del armado del ambiente")
      print("Compile el armado del ambiente y vuelva a intentarlo")
except ValueError :
  print("se ingreso un orden (tamaño de la matriz) menor a 0")
  print("ingrese un orden mayor a 0")
except pycuda.driver.LogicError:
  print("Ha ingresado un orden de la matriz igual a 0 y CUDA no puede reservar una matriz de tamaño 0")
  print("ingrese un orden para la matriz mayor a 0")
      
 

Thread x:  16 , Bloque x: 1
Thread y:  16 , Bloque y: 1
matriz original: 
[[0.83960265 0.53017443 0.36653844 0.94952106 0.2333136 ]
 [0.30120647 0.60115373 0.39224187 0.9295438  0.50992084]
 [0.6406561  0.42474523 0.8849957  0.63515824 0.6867062 ]
 [0.24683534 0.07154205 0.899879   0.12797351 0.5175436 ]
 [0.7793347  0.08413104 0.09035027 0.99284047 0.79426026]]

matriz transpuesta: 
[[0.83960265 0.30120647 0.6406561  0.24683534 0.7793347 ]
 [0.53017443 0.60115373 0.42474523 0.07154205 0.08413104]
 [0.36653844 0.39224187 0.8849957  0.899879   0.09035027]
 [0.94952106 0.9295438  0.63515824 0.12797351 0.99284047]
 [0.2333136  0.50992084 0.6867062  0.5175436  0.79426026]]

Tiempo CPU:  1.623 [ms]
Tiempo GPU:  0.304 [ms]


#4 Tabla de pasos


 Procesador | Funciòn | Detalle
------------|---------|----------
CPU      |  @param                | Lectura del tamaño del vector desde Colab.
CPU      |  import                | Importa los módulos para funcionar.
CPU      |  datetime.now()        | Toma el tiempo actual.
CPU      |  numpy.random.randn( ordenMatriz ) | Inicializa las matrices
**GPU**  |  cuda.mem_alloc()      | Reserva la memoria en GPU.
**GPU**  |  cuda.memcpy_htod()    | Copia las memorias desde el CPU al GPU.
CPU      |  SourceModule()        | Define el código del kernel donde se transpone la matriz
CPU      |  module.get_function() | Genera la función del kernel GPU
CPU      |  dim_hilo/dim_bloque   | Calcula las dimensiones.
**GPU**  |  kernel()              | Ejecuta el kernel en GPU
CPU      |  cuda.memcpy_dtoh( )   | Copia el resultado desde GPU memoria A a CPU.
CPU      |  print()               | Informo los resultados.

#5 Conclusiones

En este ejercicio que se basa en el algortimo que obtiene la matriz transpuesta de una matriz, los tiempos de ejecución resultaron más eficientes utilizando paralelismo utilizando el GPGPU porque al ser un algoritmo donde se puede mover grandes cantidades de datos ya que los tiempos de ejecución  son pequeños en comparación con la versión secuencial porque evitamos cualquier tipo de bucles, por la enorme cantidad de hilos que nos ofrece la GPU que nos permiten evitar estos bucles ya que podemos acceder de manera directa a los datos correspondientes.

Para poder sacar estas conclusiones tome 20 muestras del algoritmo en la version secuencial (CPU) y la versión con paralelismo (CPU-GPU) con un número de elementos igual a 100 y obtuve los siguientes promedios:

Promedio de tiempo ejercicio 2 secuencial: 12.224,4 ms

Promedio de tiempo de Bucle ejercicio 2: 11.497,4 ms

Promedio de tiempo CPU ejercicio 2 paralelismo: 2.407 ms 

Promedio de tiempo GPU ejercicio 2 paralelismo: 0,2676 ms


Como vemos en los resultados obtenidos, el tiempo total(CPU) en la versión con paralelismo (GPGPU) es menor al tiempo equivalente que es el tiempo de CPU, en la versión secuencial.

Por este motivo puedo decir que en los ejercicios donde tenemos que mover grandes cantidades de datos, utilizar paralelismo con (GPGPU) es más optimo porque como mencione anteriormente nos permite acceder de manera directa a los datos correspondientes y nos permite evitar el uso de bucles permitiendonos de esta manera ahorrar mucho tiempo de ejecución.

Para continuar este ejercicio tanto en la versión secuencial como en la versión con paralelismo se podría hacer que hagamos 2 movimientos de datos al mismo tiempo y ahorrar la mitad del tiempo, es decir, por ejemplo cuando se va a copiar un valor de la matriz de la posicion [ i ][ j ] a la posición [ j ][ i ],antes de hacer esto, se podria copiar el valor de la posicion [ j ][ i ] en una variable auxiliar y copiar el valor en la posicion [ i ][ j ] luego de que el valor de esta posición haya sido copiado en la posición [ j ][ i ], para poder asi, reducir el tiempo de ejecución a la mitad.

#6 Bibliografia

[1] MARKDOWN SYNTAX Colab:[PDF](https://github.com/wvaliente/SOA_HPC/blob/main/Documentos/markdown-cheatsheet-online.pdf)

[2] Introducción a Python: [pagina colab](https://github.com/wvaliente/SOA_HPC/blob/main/Documentos/Python_Basico.ipynb)

[3] Función de transponer matriz: [WEB](https://github.com/lucianopulido/EA2-Luciano-Pulido/blob/master/Bibliografia/Ejercicio%202/Fundamentos%20de%20m%C3%A9todos%20matem%C3%A1ticos%20para%20f%C3%ADsica%20e%20ingenier%C3%ADa%20.pdf)

[4] Documentación PyCUDA: [WEB](https://documen.tician.de/pycuda/index.html)

[5] Repositorio de PyCUDA: [WEB](https://pypi.org/project/pycuda/)